## 6.4 Tasky

W tej aplikacji utworzymy prostą listę pogrupowanych zadań - po raz kolejny wykorzystamy `RecyclerView` w dwoma obiektami `ViewHolder`. Całą listę będziemy zapisywać i wczytywać wz pliku za pomocą `SharedPreferences` - do zapisu posłużymy się formatrem `JSON` i biblioteką do serializacji `Gson`. Aplikacja będzie posiadała tylko jeden `Fragment` na którym umieścimy listę i prosty formularz umożliwiający dodawanie kolejnych wpisów.

<table><tr><td><img src="https://media4.giphy.com/media/Dv16lqPAAbY3wcR5yQ/giphy.gif?cid=790b76110f283c94c4653f3110e6fecb2a9d0820a3829b21&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media4.giphy.com/media/NTntAP53dVMiI7x2jO/giphy.gif?cid=790b76112a65996659d22cdbc3f09ca05eafa714f38f57d0&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media2.giphy.com/media/V1hvzLlvSPlEzKhrjM/giphy.gif?cid=790b7611f83aa25dbb5b41859105e74c7fec193a551e4e61&rid=giphy.gif&ct=g" width="200" /></td></tr></table>

### **layout**

Rozpocznijmy od layoutu aktywności do którego dodamy `Fragment`

In [ ]:
<androidx.fragment.app.FragmentContainerView android:id="@+id/fragment_main"
    android:name="pl.udu.uwr.pum.taskykotlin.TaskyFragment"
    xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="match_parent" />

oraz layoutu samego fragmentu

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    tools:context=".TaskyFragment">

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

        <LinearLayout
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:layout_margin="4dp"
            android:orientation="horizontal">

            <com.google.android.material.textfield.TextInputLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="zadanie">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/taskEditText"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:fontFamily="sans-serif"
                    android:textSize="15sp" />
            </com.google.android.material.textfield.TextInputLayout>
        </LinearLayout>

        <LinearLayout
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_margin="4dp"
            android:layout_height="wrap_content"
            android:orientation="horizontal">

            <com.google.android.material.textfield.TextInputLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="kategoria">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/groupEditText"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:fontFamily="sans-serif"
                    android:textSize="15sp" />
            </com.google.android.material.textfield.TextInputLayout>
        </LinearLayout>

        <Button
            android:id="@+id/addButton"
            android:layout_width="wrap_content"
            android:layout_gravity="center"
            android:text="dodaj"
            android:layout_margin="4dp"
            android:layout_height="wrap_content"/>
    </LinearLayout>

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/rvTasky"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:layout_margin="4dp"/>

    <Button
        android:id="@+id/clearButton"
        android:layout_width="match_parent"
        android:layout_gravity="center"
        android:layout_marginBottom="8dp"
        android:text="czyść"
        android:layout_height="wrap_content"/>

</LinearLayout>

Dodajmy również layouty dla grupy oraz pojedynczego zadania

`item_group_row.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:orientation="horizontal">

    <TextView
        android:id="@+id/groupTextView"
        android:layout_width="0dp"
        android:layout_weight="1"
        android:layout_height="wrap_content"
        android:layout_margin="4dp"
        android:text="Group"
        android:layout_marginStart="8dp"
        android:textSize="24sp" />

</LinearLayout>

`item_task_row.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content">

    <TextView
        android:id="@+id/taskTextView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        app:layout_constraintEnd_toEndOf="parent"
        android:layout_margin="4dp"
        android:layout_marginStart="8dp"
        android:textSize="18sp"
        android:paddingStart="36dp"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />


</androidx.constraintlayout.widget.ConstraintLayout>

### **model**

Zdefiniujmy model danych, będziemy posiadać dwie klasy
- `Task` - dla pojedynczego zadania
- `TaskGroup` - dla nagłówka

Na potrzeby `RecyclerView` zdefiniujemy jeszcze trzy typy
- `TaskRow` - reprezentujący rząd
- `Header` - reprezentujący nagłówek
- `Task` - reprezentujący pojedyncze zadanie

In [ ]:
sealed class TaskRow(val rowType: Int) {
    data class Task(val name: String) :
        TaskRow(TYPE_TASK)

    data class Header(val name: String, var isExpanded: Boolean = true) : TaskRow(TYPE_HEADER)

    companion object {
        const val TYPE_HEADER = 0
        const val TYPE_TASK = 1
    }
}

data class TaskGroup (val name: String)
data class Task(val name: String, val type: TaskGroup)

### **Adapter**

Przejdźmy do `TaskAdapter`

In [ ]:
class TaskAdapter(private val tasksList: MutableList<Task>) 
    : RecyclerView.Adapter<RecyclerView.ViewHolder>() {}

Zadania będziemy przechowywać w liście, w pierwszym kroku musimy pogrupować całą listę

In [ ]:
private val groupedList = tasksList.groupBy { it.type }.flatMap {
    listOf(TaskRow.Header(it.key.name), *(it.value.map { task ->
        (TaskRow.Task(task.name))
    }).toTypedArray())
}.toMutableList()

W pierwszym kroku grupujemy wszystkie zadania według type (`TYPE_HEADER` lub `TYPE_TASK`), następnie wykorzystujemy `flatMap` aby pozbyć się wewnętrznych list. W kolejny kroku tworzymy listę w której pierwszym elementem jest nagłówek, kolejnymi są wszystkie zadania z tym nagłówkiem. Na końcu zwracamy jako listę mutowalną.

Kolejnym krokiem będzie zdefiniowanie dwóch `ViewHolder` dla nagłówka i zadania.

In [ ]:
private class TaskViewHolder(private val itemBinding: ItemTaskRowBinding)
    : RecyclerView.ViewHolder(itemBinding.root) {
    fun bind(task: TaskRow.Task) {
        itemBinding.taskTextView.text = task.name
    }
}

private class HeaderViewHolder(private val itemBinding: ItemGroupRowBinding)
    : RecyclerView.ViewHolder(itemBinding.root) {
    fun bind(header: TaskRow.Header) {
        itemBinding.groupTextView.text = header.name
    }
}

Przejdźmy do metod `RecyclerView`. W `onCreateViewHolder` musimy zwrócić odpowiedni `ViewHolder`

In [ ]:
override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): RecyclerView.ViewHolder {
    return when (viewType) {
        TaskRow.TYPE_TASK -> TaskViewHolder(
            ItemTaskRowBinding.inflate(
                LayoutInflater.from(parent.context), parent, false)
        )
        TaskRow.TYPE_HEADER -> HeaderViewHolder(
            ItemGroupRowBinding.inflate(
                LayoutInflater.from(parent.context), parent, false)
        )
        else -> throw IllegalArgumentException("Wrong row type")
    }
}

Aby to zrobić musimy nadpisać metodę `getItemViewType`

In [ ]:
override fun getItemViewType(position: Int): Int = groupedList[position].rowType

`getItemCount` jest rozmiarem naszej pogrupowanej listy

In [ ]:
override fun getItemCount(): Int = groupedList.size

Na koniec dodajemy implementację metody `onBindViewHolder`.

In [ ]:
override fun onBindViewHolder(holder: RecyclerView.ViewHolder, position: Int) {
    when (val taskRow = groupedList[position]) {
        is TaskRow.Task -> (holder as TaskViewHolder).bind(taskRow)
        is TaskRow.Header -> (holder as HeaderViewHolder).bind(taskRow)
    }
}

Przejdźmy do fragmentu i dodajmy `RecyclerView`

In [ ]:
class TaskyFragment : Fragment() {

    private lateinit var binding: FragmentTaskyBinding

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentTaskyBinding.inflate(inflater, container, false)
        return binding.root
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        super.onViewCreated(view, savedInstanceState)
        val tasks: MutableList<Task> = DataProvider.dummyData.toMutableList()
        binding.rvTasky.apply {
            layoutManager = LinearLayoutManager(context)
            adapter = TaskAdapter(tasks)
        }
    }
}

W pliku `DataProvider` dodajmy dane testowe

In [ ]:
object DataProvider {
    val dummyData = listOf(
        Task("task1", TaskGroup("Group 1")),
        Task("task2", TaskGroup("Group 1")),
        Task("task3", TaskGroup("Group 1")),
        Task("task4", TaskGroup("Group 1")),
        Task("task5", TaskGroup("Group 1")),
        Task("task a", TaskGroup("Group 2")),
        Task("task b", TaskGroup("Group 2")),
        Task("task c", TaskGroup("Group 2")),
        Task("task d", TaskGroup("Group 2"))
    )
}

Możemy przetestować aplikację

<img src="https://media3.giphy.com/media/qoAHLzTQ0PezrKEhWH/giphy.gif?cid=790b7611c96fcee70976ddb748e72df1f3c57af60cc984ef&rid=giphy.gif&ct=g" width="150" />

Wróćmy do adaptera i napiszmy metodę umożliwiającą dodanie elementu. Chcemy dodać zadanie do właściwej grupy, na koniec wszystkich zadań. Potrzebujemy metodę zwracającą listę wszystkich zadań przynależących do danej grupy

In [ ]:
private fun subList(groupName: String): List<TaskRow>{
    return tasksList
        .filter { it.type.name == groupName }
        .map {TaskRow.Task(it.name)}
}

Następnie dodajmy metodę `add`

In [ ]:
fun add(task: Task, context: Context){}

W pierwszym kroku dodajmy element na listę zadań

In [ ]:
tasksList.add(task)

Znajdźmy nagłówek zadania (jeżeli istnieje)

In [ ]:
val header = groupedList
    .filterIsInstance<TaskRow.Header>()
    .find { it.name == task.type.name }

Jeżeli nagłówek nie istnieje tworzymy go i dodajemy na listę, następnie dodajemy samo zadanie i wysyłamy odpowiednie powiadomienie do adaptera

In [ ]:
if (header == null)
    groupedList.apply {
        add(TaskRow.Header(task.type.name))
        add(TaskRow.Task(task.name))
        notifyItemInserted(groupedList.size)
    }

Jeżeli nagłówek istnieje chcemy dodać zadanie na koniec podlisty

In [ ]:
else {
    val i = groupedList.indexOf(TaskRow.Header(task.type.name))
    groupedList.add(i + subList(task.type.name).size, TaskRow.Task(task.name))
    notifyItemInserted(i + subList(task.type.name).size)
}

W naszym fragmencie obsłużmy `onClick` przycisku odpowiadającego za dodanie nowego zadania.

In [ ]:
binding.addButton.setOnClickListener {
    val task = binding.taskEditText.text.toString()
    val cat = binding.groupEditText.text.toString()

    if (task.isNotEmpty() && cat.isNotEmpty()) {
        (binding.rvTasky.adapter as TaskAdapter).add(Task(task, TaskGroup(cat)), requireContext())
        binding.groupEditText.text?.clear()
        binding.taskEditText.text?.clear()
    }
}

Dodajmy również obsługę przycisku czyszczącego całą listę

In [ ]:
binding.clearButton.setOnClickListener {
    (binding.rvTasky.adapter as TaskAdapter).clear(requireContext())
}

Oraz zaimplementujmy metodę `clear` w klasie `TaskAdapter`

In [ ]:
fun clear(context: Context){
    tasksList.clear()
    notifyItemRangeRemoved(0, groupedList.size)
    groupedList.clear()
}

Możemy przetestować aplikację

<img src="https://media0.giphy.com/media/sJcy5w1ByfcZ7g50LC/giphy.gif?cid=790b761149c8246e2b3f29cede919b77aff2aff10a7990c3&rid=giphy.gif&ct=g" width="150" />

### **SharedPreferences -- JSON**

Ostatnim krokiem będzie zapisanie listy zadań do pliku. W tym celu utwórzmy plik `SharedPrefUtil` i dodajmy odpowiednie elementy. Będziemy potrzebować dwie stałe
- nazwę pliku
- klucz listy

In [ ]:
private const val TASK_LIST = "tasks"
private const val TASK_FILE = "task_file"

Chcemy skorzystać z formatu `json`, więc wykorzystamy bibliotekę `Gson` aby przygotować odpowiednio `String` do zapisu i odczytu. Musimy dodać tą bibliotekę do zależności w pliku `gradle`

In [ ]:
implementation 'com.google.code.gson:gson:2.9.0'

Następnie zaimplementujmy metodę pozwalającą zapisać dane

In [ ]:
fun saveTaskList(context: Context, list: List<Task>) {}

W pierwszym kroku utwórzmy odpowiednio sformatowany `String` za pomocą biblioteki `Gson`

In [ ]:
val json = Gson().toJson(list)

Następnie dodajmy ten `String` do `SharedPreferences` powiązanego z aplikacją

In [ ]:
val sharedPreferences = context.getSharedPreferences(TASK_FILE, MODE_PRIVATE)
sharedPreferences.edit().putString(TASK_LIST, json).apply()

Druga metoda pozwoli odczytać zapisany plik.

In [ ]:
fun getTasksList(context: Context): List<Task> {}

Po kluczu rozpakowujemy `String` z obiektu `SharedPreferences`

val sharedPreferences = context.getSharedPreferences(TASK_FILE, MODE_PRIVATE)
val json = sharedPreferences.getString(TASK_LIST, "")

Jeżeli dostaniemy `null` metoda zwraca pustą listę

In [ ]:
if (json.isNullOrEmpty()) {
    return emptyList()
}

W przeciwnym razie rozpakowujemy listę korzystając z bibllioteki `Gson`. Musimy sprecyzować co dokładnie `Gson` ma przekonwertować na listę - musimy jawnie podać typ. O ile w przypadku pojedynczych obiektów nie jest to problemem (przykładowo `Task.class`), o tyle w przypadku listy obiektów musimy wykorzystać typy generyczne czasu wykonania `TypeToken` (więcej informacji [tutaj](https://helw.net/2017/11/09/runtime-generics-in-an-erasure-world/))

In [ ]:
val type = object : TypeToken<ArrayList<Task>>() {}.type
return Gson().fromJson(json, type)

Metody wywołujemy w wewnątrz metody `add` i `clear` klasy `TaskAdapter`

In [ ]:
fun add(task: Task, context: Context){
    tasksList.add(task)
    saveTaskList(context, tasksList)
    ...
}

fun clear(context: Context){
    tasksList.clear()
    notifyItemRangeRemoved(0, groupedList.size)
    groupedList.clear()
    saveTaskList(context, tasksList)
}

oraz w `onViewCreated` klsay `TaskyFragment`

In [ ]:
val tasks: MutableList<Task> =
    getTasksList(requireContext()).toMutableList()

Możemy przetestować aplikację.

<table><tr><td><img src="https://media4.giphy.com/media/Dv16lqPAAbY3wcR5yQ/giphy.gif?cid=790b76110f283c94c4653f3110e6fecb2a9d0820a3829b21&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media4.giphy.com/media/NTntAP53dVMiI7x2jO/giphy.gif?cid=790b76112a65996659d22cdbc3f09ca05eafa714f38f57d0&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media2.giphy.com/media/V1hvzLlvSPlEzKhrjM/giphy.gif?cid=790b7611f83aa25dbb5b41859105e74c7fec193a551e4e61&rid=giphy.gif&ct=g" width="150" /></td></tr></table>